In [1]:
import numpy as np
import pandas as pd
import shapely

from src import h3_utils
import utils as U

In [2]:
df = pd.read_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bng_malls_ranked.csv")

In [4]:
df['competitor_type'] = 'primary_competitor'

In [5]:
for pid in ['129342_777438', '128050_776996']:
    df.to_csv(f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{pid}/competitors_ranked.csv",
              index=False)

In [5]:
df.to_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/mall_competitors.csv", index=False)

In [7]:
import geopandas as gpd


def get_competitors(site_id):
    def get_anchor_brands(site_id):
        g = U.get_project_info(id=site_id).query("catchment_type=='i1000mtb'")['geometry'].tolist()[0]
        g = shapely.from_wkt(g)
        stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": g}], geometry='geometry')
        coffee_shops = bng_data.query("(category=='coffee_shop') and (brand_id!='N_A')")
        coffee_shops_gdf = gpd.GeoDataFrame(coffee_shops,
                                            geometry=gpd.points_from_xy(coffee_shops['lng'], coffee_shops['lat']))
        coffee_shops_count = coffee_shops.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index()
        cat_anchor = stores_gdf.sjoin(coffee_shops_gdf, how='inner')[coffee_shops.columns.tolist()]
        cat_anchor = cat_anchor.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index()
        cat_anchor = cat_anchor.merge(coffee_shops_count, how='inner', on='brand_id')
        cat_anchor['rank'] = cat_anchor['count_y'] / cat_anchor['count_x']
        cat_anchor = cat_anchor.sort_values(by='rank', ascending=False)['brand_id'].values.tolist()[:5]
        return cat_anchor

    bng_data = pd.read_csv(
        '/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv')

    primary = bng_data.query("brand_id!='N_A' and category=='gym_fitness'")
    primary_brands = primary.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index().query(
        "count>5").sort_values(by='count', ascending=False)['brand_id'].tolist()
    df = pd.read_csv(
        f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")
    mask1 = df['brand_id'].isin(primary_brands)
    df.loc[mask1, 'competitor_type'] = 'primary_competitor'
    mask2 = (df['category'] == 'gym_fitness') & (~df['brand_id'].isin(primary_brands)) & (df['number_of_votes'] > 50)
    df.loc[mask2, 'competitor_type'] = 'secondary_competitor'
    mask3 = df['brand_id'].isin(get_anchor_brands(site_id))
    df.loc[mask3, 'competitor_type'] = 'anchor_competitor'
    df = df.query("~competitor_type.isna()")
    return df





In [3]:
import json

In [4]:
c=[{"competitor_type": "primary_competitor", "type":"brands","filter":['snapfitnessindia','stayfitindia','vivtaa','planetfitness','goldsgym','nammacrossfit']},
 {"competitor_type": "anchor_competitor","type":"brands","filter":["zomato_domino's_pizza", 'ratnadeep', 'ril', 'dominos', 'dmartindia', 'naturesbasket',
                             'marksandspencer', 'relianceretail', 'dmartreadyonline', 'naturals', 'zudio', 'lenskart', 'reliancesmartbazaar',
                             'vishalmegamart','reliancesmartpoint','toniandguy','dmart']},
 {"competitor_type": "secondary_competitor","type":"query","filter":"category.isin('gym_fitness') and number_of_votes>20 "}
 ]

In [7]:

print(json.dumps(c))

[{"competitor_type": "anchor_competitor", "type": "brands", "filter": ["blown", "bouncehere", "bblunt", "nailashes", "nykdbynykaa", "sugarcosmetics", "globaldesi", "naturals", "shopforaurelia", "biba", "yesmadam", "rangriti", "nykaa", "nailbox", "bibaindia", "fabindia"]}, {"competitor_type": "secondary_competitor", "type": "brands", "filter": ["forestessentialsindia", "colorbarcosmetics", "lakmeindia", "lakmesalon"]}, {"competitor_type": "secondary_competitor", "type": "query", "filter": "category.isin(['cosmetic']) "}, {"competitor_type": "primary_competitor", "type": "brands", "filter": ["jockeyindia", "zivame", "nykdbynykaa", "enamor", "adityabirla", "vanheusenindia", "jockey"]}]


In [6]:
c = [{'competitor_type': "anchor_competitor", "type": 'brands',
      "filter": ['blown', 'bouncehere', 'bblunt', 'nailashes', 'nykdbynykaa', 'sugarcosmetics', 'globaldesi',
                 'naturals', 'shopforaurelia', 'biba', 'yesmadam', 'rangriti', 'nykaa', 'nailbox',
                 'bibaindia', 'fabindia']},
     {"competitor_type": "secondary_competitor", "type": "brands",
      "filter": ['forestessentialsindia', 'colorbarcosmetics', 'lakmeindia', 'lakmesalon', ]},
     {"competitor_type": "secondary_competitor", "type": "query",
      "filter": "category.isin(['cosmetic']) "},
     {"competitor_type": "primary_competitor", "type": "brands",
      "filter": ['jockeyindia', 'zivame', 'nykdbynykaa', 'enamor', 'adityabirla', 'vanheusenindia',
                 'jockey']}]

In [7]:
def get_competitors(site_id):
    df = pd.read_csv(
        f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")
    c = [{'competitor_type': "anchor_competitor", "type": 'brands',
      "filter": ['blown', 'bouncehere', 'bblunt', 'nailashes', 'nykdbynykaa', 'sugarcosmetics', 'globaldesi',
                 'naturals', 'shopforaurelia', 'biba', 'yesmadam', 'rangriti', 'nykaa', 'nailbox',
                 'bibaindia', 'fabindia']},
     {"competitor_type": "secondary_competitor", "type": "brands",
      "filter": ['forestessentialsindia', 'colorbarcosmetics', 'lakmeindia', 'lakmesalon', ]},
     {"competitor_type": "secondary_competitor", "type": "query",
      "filter": "category.isin(['cosmetic']) "},
     {"competitor_type": "primary_competitor", "type": "brands",
      "filter": ['jockeyindia', 'zivame', 'nykdbynykaa', 'enamor', 'adityabirla', 'vanheusenindia',
                 'jockey']}]
    for f in c:
        if f['type'] == "brands":
            mask=df.query("brand_id.isin(@f['filter'])")
        elif f['type'] == "query":
            mask = df.query(f['filter'])
        else:
            raise ValueError("Filter Not Supported")
        df.loc[mask.index, 'competitor_type'] = f['competitor_type']
    df = df.query("~competitor_type.isna()")
    return df

In [8]:
sids = ['129696_776307', '130959_775791']
for sid in sids[:]:
    df = get_competitors(sid)
    df.to_csv(f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{sid}/competitors.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
get_competitors(sids[0])

<IPython.core.display.Javascript object>

,id,name,lat,lng,cid,brand_id,brand_name,number_of_votes,rating,place_url,source,category,top_category,type,created_at,competitor_type
33,ChIJjUSe76YXrjsRlxcXfVzhzGk,Nailashes Indiranagar,12.970464,77.643417,0x3bae17a6ef9e448d:0x69cce15c7d171797,nailashes,Nailashes,126,4.8,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,salon,service,N_A,1699615207,anchor_competitor
149,ChIJEUrGz6kWrjsRDK-d0hNuV-I,"BBLUNT Salon - Indiranagar, Bangalore",12.970244,77.638924,0x3bae16a9cfc64a11:0xe2576e13d29daf0c,bblunt,BBlunt,1193,4.6,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,salon,service,N_A,1699615207,anchor_competitor
175,ChIJAXEwvk8XrjsRBSLNe4xcIOE,Jockey Exclusive Store,12.978196,77.637688,0x3bae174fbe307101:0xe1205c8c7bcd2205,jockey,Jockey,190,4.8,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,clothing_store,retail,N_A,1687847039,primary_competitor
177,ChIJHRA8W0AXrjsRSD2KU7VgJEE,"Fabindia Store (Indiranagar, Bangalore)",12.969950,77.639053,0x3bae17405b3c101d:0x412460b5538a3d48,fabindia,Fabindia,277,4.3,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,clothing_store,retail,N_A,1695212099,anchor_competitor
260,ChIJG5KZppwWrjsRHV4uw1LKHNk,Clinique,12.973547,77.620438,0x3bae169ca699921b:0xd91cca52c32e5e1d,clinique,Clinique,5,3.6,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,cosmetic,retail,N_A,1687847039,secondary_competitor
432,ChIJBU-fnRwUrjsRpLXLQIWSRec,lakme salon,12.963832,77.640205,0x3bae141c9d9f4f05:0xe745928540cbb5a4,lakmeindia,Lakme,1437,4.6,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,salon,service,N_A,1699615207,secondary_competitor
899,ChIJm6AoPagWrjsRwZvz-tNPvQk,Lakme,12.970327,77.639702,0x3bae16a83d28a09b:0x9bd4fd3faf39bc1,lakmeindia,Lakme,1714,4.7,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,salon,service,N_A,1699615207,secondary_competitor
1030,ChIJr86vo5wWrjsR0NPjtfjBHtU,Fabindia,12.973771,77.623001,0x3bae169ca3afceaf:0xd51ec1f8b5e3d3d0,fabindia,Fabindia,96,4.3,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,clothing_store,retail,N_A,1695212099,anchor_competitor
1237,ChIJG6Xc-acWrjsR1-Tq2jF0Zjw,Aurelia,12.969621,77.641487,0x3bae16a7f9dca51b:0x3c667431daeae4d7,shopforaurelia,Aurelia,48,3.6,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,clothing_store,retail,N_A,1687847039,anchor_competitor
1325,ChIJXfklb34XrjsRfdaa2Zhp5Dk,Fabindia,12.972281,77.618675,0x3bae177e6f25f95d:0x39e46998d99ad67d,fabindia,Fabindia,206,4.3,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,clothing_store,retail,N_A,1695212099,anchor_competitor


In [69]:
df.to_csv(f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{sid}/competitors.csv")

In [13]:
bng_data = pd.read_csv(
    '/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv')

In [14]:
primary = bng_data.query("brand_id!='N_A' and category=='gym_fitness'")
primary_brands = primary.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index().query(
    "count>5").sort_values(by='count', ascending=False)


In [21]:
secondary_brands = bng_data.query(
    "category=='gym_fitness' and number_of_votes>50 and brand_id not in(@primary_brands['brand_id']) ")
secondary_brands

,id,name,lat,lng,cid,brand_id,brand_name,number_of_votes,place_url,source,category,top_category,type,created_at
6,ChIJawgheE0TrjsRjbLaEk_6Euo,Barbell Cartel Gym,12.957161,77.736313,0x3bae134d7821086b:0xea12fa4f12dab28d,N_A,N_A,167,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
231,ChIJkdKgmYA9rjsRY-ml6pDMNAI,FEMININE WOMEN'S ONLY GYM Nagarabhavi,12.960160,77.506882,0x3bae3d8099a0d291:0x234cc90eaa5e963,N_A,N_A,62,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
344,ChIJubahgNoXrjsRT5pniThZZjA,SUPRAD Badminton Academy,13.050083,77.607834,0x3bae17da80a1b6b9:0x3066593889679a4f,N_A,N_A,192,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
485,ChIJlbPzOWkRrjsRCol-7vLx-wI,DRDO Phase II Sports Complex,12.986635,77.681442,0x3bae116939f3b395:0x2fbf1f2ee7e890a,N_A,N_A,391,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
593,ChIJs4uw8y0TrjsRlEGp806yPz0,The Professional Fitness Center (Team:TPF) ದಿ ...,12.904058,77.676003,0x3bae132df3b08bb3:0x3d3fb24ef3a94194,N_A,N_A,264,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362156,ChIJDZQzV50VrjsRfh_J06vTLLk,LL PG for MEN,12.953691,77.578125,0x3bae159d5733940d:0xb92cd3abd3c91f7e,N_A,N_A,53,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362165,ChIJL2Sp-TAVrjsRMl4htzgNf8M,GOLDEN FIZZIK GYM,12.928550,77.596794,0x3bae1530f9a9642f:0xc37f0d38b7215e32,N_A,N_A,209,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362306,ChIJ965kur8NrjsRDfcAzoSTxxY,GET FIT WITH SHEHI (Personal Trainer),12.989023,77.754524,0x3bae0dbfba64aef7:0x16c79384ce00f70d,N_A,N_A,118,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362323,ChIJn5eOvwoXrjsR7U0C4mhV0Kg,Refuel fitness studio,13.014141,77.639542,0x3bae170abf8e979f:0xa8d05568e2024ded,N_A,N_A,63,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039


In [23]:
anchor = bng_data.query("(category=='coffee_shop') and (brand_id!='N_A')").groupby('brand_id').agg(
    count=("id", np.count_nonzero)).reset_index()
anchor

,brand_id,count
0,1by2foods,1
1,3mangos,3
2,amrutchaha,1
3,artscafe,3
4,avoke,3
...,...,...
120,tribalbrew,3
121,vaamanamruttulya,1
122,vidyacoffee,7
123,wafl,1


In [26]:
site_id = '129065_775925'
df = pd.read_csv(
    f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")

In [30]:
cat_anchor = df.query("(category=='coffee_shop') and (brand_id!='N_A')").groupby('brand_id').agg(
    count=("id", np.count_nonzero)).reset_index()
cat_anchor = cat_anchor.merge(anchor, how='inner', on='brand_id')
cat_anchor['rank'] = cat_anchor['count_y'] / cat_anchor['count_x']
cat_anchor = cat_anchor.sort_values(by='rank', ascending=False)
cat_anchor


,brand_id,count_x,count_y,rank
18,hattikaapi,2,29,14.500000
36,slay,1,12,12.000000
24,lassishop,3,31,10.333333
37,smoor,1,10,10.000000
1,bluetokaicoffee,1,10,10.000000
6,chaayos,3,29,9.666667
22,lassicorner,4,38,9.500000
3,cafecoffeeday,14,129,9.214286
10,chaipoint,10,83,8.300000
42,teabarind,1,8,8.000000
